In [1]:
# %load PCA.py
%matplotlib inline
import pandas as pd
import numpy as np
import re
import os
import PyPDF2
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from gensim.models.word2vec import Word2Vec



Using TensorFlow backend.


In [3]:
stemmer=SnowballStemmer("english")
def pdf_to_text(pdf):

    pdfReader = PyPDF2.PdfFileReader(pdf)
    numPage=pdfReader.numPages
    pdftext=" "
    for page in range(numPage):
        pageObj =pdfReader.getPage(page)
        text= pageObj.extractText()
        pagetext="".join(text)
        pdftext=" ".join([pagetext, pdftext])
    alltext = re.sub('[\n]', '', pdftext)
    all_words=alltext.split(" ")
    affterstemmer=[]
    for word in all_words:
        affterstemmer.append(stemmer.stem(word))
    stop = set(stopwords.words('english'))
    afterstop=[]
    for word in affterstemmer:
        if word not in stop:
            afterstop.append(word)
    return " ".join(afterstop)
THIS_FOLDER = os.path.dirname(os.path.abspath("Word2Vec.ipynb"))


## Geting ready the training data
data = pd.read_csv('resumes.csv')




In [5]:
X_train=[]
for i in data['path']:
    my_file = os.path.join(THIS_FOLDER, i)
    pdf = open(my_file, 'rb')
    X_train.append(pdf_to_text(pdf))



In [ ]:
word2vec = {}

In [49]:
x_by_words=[]
for i in X_train:
    x_by_words.append(i.split())

In [187]:
s = 500

model = Word2Vec(x_by_words, size=s, min_count=5, workers=10)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}
vocab = model.wv.index2word
train_w2v = []
for i in x_by_words:
    ar =[0 for i in range(s)]
    for j in i:
        if j in vocab:
            ar+=w2v[j]
    train_w2v.append(ar)

In [188]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_w2v, data['succes'], test_size = .001, random_state = 32)

In [189]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

In [190]:
clfs=[GaussianNB(),SGDClassifier(),LogisticRegression(),
      DecisionTreeClassifier(),RandomForestClassifier(),
      GradientBoostingClassifier(),AdaBoostClassifier()]
for c in clfs:
    clf = c.fit(x_train,y_train)
    print(np.mean(cross_val_score(clf, train_w2v, data['succes'], cv=3)))



0.737406689003
0.731381884441
0.940486686651
0.849908893199
0.871833303944
0.893640157527
0.903074119791


In [191]:

clf = LogisticRegression()
cvs = cross_val_score(clf, x_train, y_train, cv=5)
print(cvs)
print(np.mean(cvs))

[ 0.92307692  0.96875     0.90625     0.95238095  0.92063492]
0.934218559219
